In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
Load the data set we will be working with
"""

data_dir = '../../Datasets'

df = pd.read_csv(f'{data_dir}/study_213/top_5_213.csv')
df.drop(columns = 'DATE', inplace = True)
display(df)

,7058,8572,39601,8901,7730
0,1531.0,422.0,19.0,164.0,2.0
1,437.0,94.0,17.0,97.0,1.0
2,189.0,66.0,36.0,23.0,761.0
3,370.0,116.0,83.0,196.0,561.0
4,101.0,5.0,21.0,233.0,2.0
...,...,...,...,...,...
1510,255.0,16.0,185.0,44.0,15.0
1511,741.0,735.0,252.0,87.0,382.0
1512,287.0,291.0,14.0,16.0,1.0
1513,732.0,468.0,32.0,186.0,1.0


In [3]:
"""
Set the parameters for the linear regression here
"""

# We will look at the last three values for each species and try to predict the next value using linear regression
lag = 10

expected = df.iloc[lag:, :]
display(expected)

,7058,8572,39601,8901,7730
10,505.0,741.0,70.0,378.0,173.0
11,174.0,233.0,48.0,58.0,1363.0
12,250.0,11.0,2131.0,19.0,196.0
13,1002.0,3.0,860.0,32.0,36.0
14,105.0,2670.0,2257.0,11.0,66.0
...,...,...,...,...,...
1510,255.0,16.0,185.0,44.0,15.0
1511,741.0,735.0,252.0,87.0,382.0
1512,287.0,291.0,14.0,16.0,1.0
1513,732.0,468.0,32.0,186.0,1.0


In [4]:
def predict(index, species, df, lag):
    data = df.loc[index - lag : index - 1, species]

    y = data.to_numpy()
    x = np.arange(start = index - lag, stop = index)
    
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    
    return m * index + c


In [5]:
predicted = np.array([[predict(index, species, df, lag) for index in range(lag, len(df))] for species in df.columns])

In [6]:
error = pd.DataFrame()

for i, column in enumerate(df.columns):
    error[column] = pd.Series((expected.to_numpy().T - predicted)[i])

display(error)

,7058,8572,39601,8901,7730
0,-210.666667,648.933333,-1695.533333,271.000000,49.400000
1,-737.733333,-171.933333,-1346.266667,-163.200000,1301.266667
2,-502.066667,-394.866667,1120.733333,-156.266667,-268.200000
3,427.800000,-298.266667,-602.466667,-64.600000,-475.733333
4,-636.333333,2459.533333,977.866667,-58.533333,-413.600000
...,...,...,...,...,...
1500,-37.000000,-534.933333,-582.133333,-15.666667,-113.800000
1501,422.266667,373.200000,-105.800000,22.400000,339.600000
1502,-167.200000,-165.133333,-30.733333,-36.533333,-117.533333
1503,388.200000,173.533333,438.466667,170.933333,-31.933333
